In [ ]:
# softmax 从 0 实现

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

## 1. 初始化模型参数
输入：原始数据集中的每个样本都是 $28*28$ 的图像。 本节将展平每个图像，把它们看作长度为 $784$ 的向量。
输出：数据集有10个类别，所以网络输出维度为10。
权重：用正态分布初始化.权重将构成一个 $784*10$ 的矩阵
偏置：全 0 初始化,偏置将构成一个 $1*10$ 的行向量

In [2]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

## 2. 定义softmax操作
 softmax 可以确保概率和 = 1

In [4]:
# 回顾 sum() :
#   dim=0,上下挤压
#   dim=1,左右挤压

def softmax(X):
    X_exp = torch.exp(X)  # 对 Tensor 中的每个项求幂（即 x->e^x）
    partition = X_exp.sum(1, keepdim=True)  # 求每一行的和（小批量中每个样本是一行），得到每个样本的规范化常数；
    return X_exp / partition  # 将每一行除以其规范化常数，确保结果的和为1。这里应用了广播机制

测试上面的 softmax 函数

In [5]:
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X,X_prob, X_prob.sum(1)

(tensor([[-0.3800, -0.8959, -1.9992, -0.1317,  1.1706],
         [-1.0616, -0.6667, -0.4951,  0.5617,  0.3676]]),
 tensor([[0.1283, 0.0766, 0.0254, 0.1645, 0.6051],
         [0.0741, 0.1100, 0.1306, 0.3758, 0.3095]]),
 tensor([1.0000, 1.0000]))

## 3.定义模型
注意， 将数据传递到模型之前，我们使用reshape函数将每张原始图像（28\*28）展平为向量（1\*784）。

In [6]:
def net(X):
    # torch.matmul(X,W):  [1,784] * [784,10] -> [1,10]
    # torch.matmul() + b: [1,10] + [1,10]
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

## 4.定义损失函数
交叉熵损失函数，这可能是深度学习中最常见的损失函数。

In [7]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])

# 索引数组要比 Python for 循环来得高效
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y]) # 索引数组.range(len(y_hat))表示遍历每一行, y表示指定列

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

## 5. 分类精度
做分类时可能在内部估计概率，但最终必须在类别中选择一个。
首先，如果 `y_hat` 是矩阵，那么假定第二个维度(每一列)存储每个类的预测分数。
我们使用 `argmax` 获得每行中最大元素的索引来获得预测类别。
然后将预测类别与真实y元素进行比较。
由于等式运算符“==”对数据类型很敏感， 因此我们将 `y_hat` 的数据类型转换为与 `y` 的数据类型一致。 结果是一个包含0（错）和1（对）的张量。 最后，我们求和会得到正确预测的数量。

In [8]:
def accuracy(y_hat, y):
    if len(y_hat.shape)>1 and y_hat.shape[1]>1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [9]:
accuracy(y_hat, y) / len(y)

0.5

##  6. 训练

## 7. 预测